In [150]:
import matplotlib.pyplot as plt

from turtle import forward
import numpy as np
import pandas as pd
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

from torch.utils.data import Dataset, DataLoader

import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
from scipy.fft import fft

from pathlib import Path
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms  

import torch.nn.functional as F





from sklearn.metrics import accuracy_score, confusion_matrix


def acc_m(test_dataset, model):
    device = next(model.parameters()).device
    y_test = []
    y_test_hat = []
    model.eval()
    
    for inputs, labels in test_dataset:
        # Add batch dimension like in training
        inputs = inputs.unsqueeze(0)  # [freq, time] -> [1, freq, time]
        
        with torch.no_grad():
            outputs = model(inputs.to(device))
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
        
        y_test.append(labels)  # labels is already an int
        y_test_hat.extend(preds)

    acc = accuracy_score(y_test, y_test_hat)
    print(f'Accuracy: {acc*100:.2f} %')
    cm = confusion_matrix(y_test, y_test_hat)
    print(cm)

In [152]:


class AudioFolder(Dataset):
    def __init__(self, root, transform=None, target_transform=None,
                 extensions=(".wav", ".flac", ".mp3")):
        self.root = Path(root)
        self.transform = transform
        self.target_transform = target_transform
        self.extensions = extensions

        # crawl sub-dirs and record (path, class_idx)
        self.classes = sorted({p.parent.name for p in self.root.rglob("*") if p.suffix in extensions})
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.samples = [(p, self.class_to_idx[p.parent.name])
                        for p in self.root.rglob("*") if p.suffix in extensions]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        waveform, sr = torchaudio.load(path, backend="ffmpeg")

        if self.transform:
            waveform = self.transform(waveform)
        if self.target_transform:
            label = self.target_transform(label)

        return waveform[0], label


transform_wav = lambda x: torchaudio.functional.spectrogram(
        x,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )
dataset_wav = AudioFolder('../data/hb/', transform_wav)
dataset_wav_train, dataset_wav_test = train_test_split(dataset_wav, train_size=0.9)


class AudioClassifier(nn.Module):
    def __init__(self, n_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        def build_Seq():
            return nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            # nn.BatchNorm2d(16),
            nn.Conv2d(32, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            )
        self.real = build_Seq()

        self.imag = build_Seq()

        in_size = 24000 * 2
        self.fc = nn.Sequential(
            nn.Linear(in_size, in_size // 8),
            nn.Linear(in_size // 8, in_size // 32),
            nn.Linear(in_size // 32, n_classes)
        )

    def forward(self, x):
        real_input = x.real.unsqueeze(1)
        img_input = x.imag.unsqueeze(1)

        out_l = self.real(real_input)   
        out_r = self.imag(img_input)        

        out_l = F.adaptive_avg_pool2d(out_l, (30, 100))  # Fixed reasonable size
        out_r = F.adaptive_avg_pool2d(out_r, (30, 100))  # [batch, 8, 30, 100]

        out_l = out_l.flatten(1)  # [batch, 24000]
        out_r = out_r.flatten(1)  # [batch, 24000]
        out = torch.cat([out_l, out_r], dim=1)  # [batch, 2*channels*freq*time]
        return self.fc(out)
    
ac = AudioClassifier(len(dataset_wav.classes))


In [59]:
import random
data_path = '../data/hb/'

for i in range(10):
    unl = 'unlabelled'
    unlab = Path(data_path,unl)
    random_unlabel = random.choice(list(unlab.glob('*')))
    waveform, sr = torchaudio.load(random_unlabel, backend="ffmpeg")
    transformed = fft(waveform.reshape(-1).numpy())
    print(transformed.shape)
# STFT: windowing + overlap + FFT all in one
    spec = torchaudio.functional.spectrogram(
        waveform,
        pad=0,
        win_length = 255,
        window=torch.hann_window(255),  # Hann window
        n_fft=255,
        normalized=False,
        center=False ,
        hop_length=128,                  # overlap = n_fft - hop_length
        power=None                       # None -> keep complex, 2.0 -> magnitude^2
    )
    print(spec.shape)    

(41625,)
torch.Size([1, 128, 324])
(48057,)
torch.Size([1, 128, 374])
(49400,)
torch.Size([1, 128, 384])
(47952,)
torch.Size([1, 128, 373])
(47952,)
torch.Size([1, 128, 373])
(60968,)
torch.Size([1, 128, 475])
(48057,)
torch.Size([1, 128, 374])
(50145,)
torch.Size([1, 128, 390])
(47092,)
torch.Size([1, 128, 366])
(47092,)
torch.Size([1, 128, 366])


In [141]:
sm = dataset_wav_test.pop

In [145]:
# sm = sm()
sm[0].shape

torch.Size([128, 428])

In [153]:

train_dataset = dataset_wav_train
test_dataset = dataset_wav_test

loss = nn.CrossEntropyLoss()
device = 'cuda'
ac = ac.to(device)
optimiser = torch.optim.Adam(ac.parameters(), lr=0.0001)

NUM_EPOCHS = 20
losses = []

for epoch in range(NUM_EPOCHS):
    for i, (inputs, labels) in enumerate(train_dataset):
        optimiser.zero_grad()
        
        # Add batch dimension manually
        inputs = inputs.unsqueeze(0)  # [1, freq, time]
        labels = torch.tensor([labels])  # [1]
        
        y_pred = ac(inputs.to(device))
        loss_v = loss(y_pred, labels.to(device))
        
        loss_v.backward()
        losses.append(loss_v.item())
        optimiser.step()
        
        if i % 50 == 0:
            print(f'Epoch {epoch}/{NUM_EPOCHS}, Sample {i}, Loss: {loss_v.item():.4f}')
    
    print(f'Completed Epoch {epoch+1}')
    acc_m(test_dataset, ac)

Epoch 0/20, Sample 0, Loss: 1.7950
Epoch 0/20, Sample 50, Loss: 0.8155
Epoch 0/20, Sample 100, Loss: 1.5056
Epoch 0/20, Sample 150, Loss: 1.7920
Completed Epoch 1
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 1/20, Sample 0, Loss: 1.6342
Epoch 1/20, Sample 50, Loss: 1.6106
Epoch 1/20, Sample 100, Loss: 1.6471
Epoch 1/20, Sample 150, Loss: 1.8081
Completed Epoch 2
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 2/20, Sample 0, Loss: 1.5690
Epoch 2/20, Sample 50, Loss: 1.4755
Epoch 2/20, Sample 100, Loss: 1.6046
Epoch 2/20, Sample 150, Loss: 1.7318
Completed Epoch 3
Accuracy: 16.67 %
[[0 0 0 2 0]
 [0 0 0 1 0]
 [0 0 0 7 0]
 [0 0 0 3 0]
 [0 0 0 5 0]]
Epoch 3/20, Sample 0, Loss: 1.5456
Epoch 3/20, Sample 50, Loss: 1.3942
Epoch 3/20, Sample 100, Loss: 1.2745
Epoch 3/20, Sample 150, Loss: 1.9595
Completed Epoch 4
Accuracy: 27.78 %
[[0 0 0 0 2]
 [0 0 0 0 1]
 [0 0 0 0 7]
 [0 0 0 0 3]
 [0 0 0 0 5]]
Epoch 4/20, 